In [1]:
import pandas as pd

df = pd.read_json('./snli_1.0/snli_1.0_train.jsonl', lines=True)
df = df[['captionID', 'sentence1', 'sentence2']]

In [ ]:
prems = df.drop_duplicates(subset=['captionID'])['sentence1']
hyps = df['sentence2'].drop_duplicates()



0         A person on a horse jumps over a broken down a...
3                     Children smiling and waving at camera
6         A boy is jumping on skateboard in the middle o...
9         An older man sits with his orange juice at a s...
12                 Two blond women are hugging one another.
                                ...                        
550137    A group of four kids stand in front of a statu...
550140       a kid doing tricks on a skateboard on a bridge
550143         A dog with a blue collar plays ball outside.
550146                  Four dirty and barefooted children.
550149    A man is surfing in a bodysuit in beautiful bl...
Name: sentence1, Length: 151196, dtype: object


In [26]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter, defaultdict
import math

nltk.download('stopwords')
nltk.download('punkt_tab')

stopwords = set(stopwords.words('english'))

def get_pmi(target, corpus, stopwords, threshold=10):
    N = len(corpus)
    target_count = 0
    single_count = Counter()
    pair_count = Counter()

    pair_pmi = dict()
    pair_location = defaultdict(list) 
    
    for idx, text in enumerate(corpus):
        tokens = word_tokenize(text.lower())
        words = set()
        for tok in tokens:
            if tok.isalpha() and tok not in stopwords:
                words.add(tok)

        contain_target = target in words
 
        for w in words:
            single_count[w] += 1
            if contain_target:
                pair_count[w] += 1 
                pair_location[w].append(idx)
        target_count += 1 if contain_target else 0
    
    if target_count == 0:
        return None 
    
    for w in pair_count:
        if single_count[w] >= threshold:
            pmi = math.log2(N * pair_count[w]) - math.log2(target_count * single_count[w])
            pair_pmi[w] = pmi 

    return pair_pmi, pair_location


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\brian\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\brian\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
prems_pair_pmi, prems_pair_location = get_pmi("woman", prems, stopwords)

In [39]:
pmi_list = [(w, pmi) for w, pmi in prems_pair_pmi.items()]
sorted_list = sorted(pmi_list, key=lambda x: x[1], reverse=True)
print(sorted_list)

[('woman', 2.892752231523332), ('mascara', 2.892752231523332), ('lipstick', 2.5931919496644262), ('incense', 2.57082413663597), ('veil', 2.530182152138625), ('clothesline', 2.4452932545521158), ('headscarf', 2.4420908225137694), ('sari', 2.4333206128860354), ('blouse', 2.4167241023553423), ('purse', 2.3868613017933775), ('hijab', 2.3781790586935756), ('handbag', 2.32571163879944), ('shawl', 2.3233865858531964), ('pregnant', 2.3077897308021775), ('applied', 2.3077897308021775), ('leopard', 2.2760808710748393), ('revealing', 2.2647210089102927), ('loom', 2.260484016023824), ('chats', 2.240675534943641), ('creme', 2.240675534943641), ('mushroom', 2.240675534943641), ('bra', 2.2297872188009045), ('applying', 2.21852239270509), ('applies', 2.214680326410697), ('earrings', 2.192312513382241), ('jars', 2.192312513382241), ('sparkly', 2.192312513382241), ('skirt', 2.186774329840812), ('heels', 2.184518355900444), ('drawer', 2.155786637357128), ('claw', 2.155786637357128), ('sunhat', 2.15578663